In [1]:
import pandas as pd
import json
import statistics
import numpy as np
import datetime as dt
import iexfinance as iexf
import pyprind as bar

In [2]:
start = dt.date(2018,5,15)
end = dt.date(dt.date.today().year, dt.date.today().month,dt.date.today().day-1)
end_iexf = dt.date(dt.date.today().year, dt.date.today().month,dt.date.today().day-2)
end_t = dt.datetime(end.year,end.month,end.day-1, 4,0,0)

In [3]:
stock_symbols = ["MSFT",
                "INTC",
                "ORCL",
                "UPS",
                "FDX",
                "XOM",
                "BP"
                ]
stock_descriptions = {"MSFT":"Microsoft Corporation",
                      "INTC":"Intel Corporation",
                      "ORCL":"Oracle Corporation",
                      "UPS":"United Parcel Service Inc.",
                      "FDX":"FedEx Corporation",
                      "XOM":"Exxon Mobil Corporation",
                      "BP":"BP p.l.c."
                     }

#### Yahoo! Finance Data

In [4]:
start_utc = int(dt.datetime(2018,5,15,4,0,0).timestamp())
end_utc = int(end_t.timestamp())

yahoo_url = "https://finance.yahoo.com/quote/{0}/history?period1={1}&period2={2}&interval=1d&filter=history&frequency=1d"

yahoo_col_data = {"Date":"Date",
                  "Open":"Open",
                  "High":"High",
                  "Low":"Low",
                  "Close*":"Close",
                  "Adj Close**":"AdjClose",
                  "Volume":"Volume",
                  "Symbol":"Symbol"
}
yahoo_col_dtypes = {"Date":"datetime64[ns]",
                    "Open":"float64",
                    "High":"float64",
                    "Low":"float64",
                    "Close":"float64",
                    "AdjClose":"float64",
                    "Volume":"float64",
                    "Symbol":"object"
                   }


In [5]:
progress = bar.ProgBar(len(stock_symbols), monitor=True, title="Scraping Yahoo! Finance Historical Prices")
for sym in stock_symbols:
    label = sym.replace("^","")
    temp = pd.read_html(yahoo_url.format(sym,start_utc,end_utc))
    df = temp[0][:-1].copy()
    df["Symbol"] = label
    df.rename(columns=yahoo_col_data,inplace=True)
    df.replace(to_replace = "-", value = 0, inplace=True)
    df.drop_duplicates("Date", keep="first", inplace=True)
    df = df.astype(yahoo_col_dtypes)
    df = df.drop(columns="AdjClose")
    globals()["yahoo_%s" %label] = df
    globals()["yahoo_%s_df" %label] = globals()["yahoo_%s" %label].set_index("Date")
    globals()["yahoo_%s_df" %label].index = pd.to_datetime(globals()["yahoo_%s_df" %label].index)
    globals()["yahoo_%s_df" %label].sort_index(ascending = True, inplace = True)
    progress.update()

Scraping Yahoo! Finance Historical Prices
0% [#######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


#### IEX Financial Data

In [6]:
iex_col_data = ["Date","Open","High","Low","Close","Volume","Symbol"]
iex_col_dtypes = {"Date":"datetime64[ns]",
                  "Open":"float64",
                  "High":"float64",
                  "Low":"float64",
                  "Close":"float64",
                  "Volume":"float64",
                  "Symbol":"object"
                 }

In [7]:
for stock in stock_symbols:
    globals()["iex_%s" %stock] = iexf.get_historical_data(stock, start=start, end=end_iexf)
    temp_date = []
    temp_open = []
    temp_high = []
    temp_low = []
    temp_close = []
    temp_vol = []
    temp_symbol = []

    temp_keys = globals()["iex_%s" %stock][stock]
    for key in temp_keys.keys():
        temp_date.append(key)
        temp_open.append(temp_keys[key]["open"])
        temp_high.append(temp_keys[key]["high"])
        temp_low.append(temp_keys[key]["low"])
        temp_close.append(temp_keys[key]["close"])
        temp_vol.append(temp_keys[key]["volume"])
        temp_symbol.append(stock)
    globals()["iex_%s" %stock] = pd.DataFrame(np.column_stack([temp_date,temp_open, temp_high,temp_low,temp_close,temp_vol,temp_symbol]),
                          columns=iex_col_data).astype(iex_col_dtypes)
    globals()["iex_%s_df" %stock] = globals()["iex_%s" %stock].set_index("Date")


##### Yahoo! Finance DataFrames with Date Index

In [8]:
yahoo_BP_df

,Open,High,Low,Close,Volume,Symbol
Date,,,,,,
2018-05-15,46.51,46.99,46.27,46.79,7149600.0,BP
2018-05-16,46.80,46.82,46.43,46.71,4866500.0,BP
2018-05-17,47.01,47.37,46.93,47.19,7207900.0,BP
2018-05-18,47.30,47.44,47.14,47.38,4276600.0,BP
2018-05-21,47.38,47.83,47.38,47.79,5412100.0,BP
2018-05-22,47.37,47.71,46.93,47.09,7979900.0,BP
2018-05-23,46.13,46.51,45.90,46.45,7839900.0,BP
2018-05-24,45.67,45.94,45.40,45.75,7729200.0,BP
2018-05-25,44.82,44.83,44.09,44.35,9171300.0,BP


In [9]:
yahoo_FDX_df

,Open,High,Low,Close,Volume,Symbol
Date,,,,,,
2018-05-15,249.48,249.58,246.25,247.34,1638200.0,FDX
2018-05-16,247.78,248.72,246.54,247.12,1101200.0,FDX
2018-05-17,247.69,250.23,247.28,248.46,1084000.0,FDX
2018-05-18,247.88,249.73,246.71,249.45,1195700.0,FDX
2018-05-21,251.00,254.94,250.68,254.66,1384200.0,FDX
2018-05-22,255.15,256.30,252.35,252.59,1011300.0,FDX
2018-05-23,252.02,252.02,248.62,250.47,1095500.0,FDX
2018-05-24,250.18,252.01,248.50,251.74,1399400.0,FDX
2018-05-25,250.86,253.18,250.86,251.50,841200.0,FDX


In [10]:
yahoo_INTC_df

,Open,High,Low,Close,Volume,Symbol
Date,,,,,,
2018-05-15,54.58,54.75,53.56,53.92,21712600.0,INTC
2018-05-16,54.17,54.70,53.96,54.64,16260500.0,INTC
2018-05-17,54.55,55.44,54.42,54.81,16872600.0,INTC
2018-05-18,54.53,54.54,53.47,53.50,23679900.0,INTC
2018-05-21,54.36,55.22,53.86,54.32,25367400.0,INTC
2018-05-22,54.73,55.05,54.05,54.45,19009600.0,INTC
2018-05-23,54.00,55.23,53.84,55.21,20496500.0,INTC
2018-05-24,55.18,55.30,54.13,54.75,19443000.0,INTC
2018-05-25,54.80,55.50,54.54,55.44,18550000.0,INTC


In [11]:
yahoo_INTC_df

,Open,High,Low,Close,Volume,Symbol
Date,,,,,,
2018-05-15,54.58,54.75,53.56,53.92,21712600.0,INTC
2018-05-16,54.17,54.70,53.96,54.64,16260500.0,INTC
2018-05-17,54.55,55.44,54.42,54.81,16872600.0,INTC
2018-05-18,54.53,54.54,53.47,53.50,23679900.0,INTC
2018-05-21,54.36,55.22,53.86,54.32,25367400.0,INTC
2018-05-22,54.73,55.05,54.05,54.45,19009600.0,INTC
2018-05-23,54.00,55.23,53.84,55.21,20496500.0,INTC
2018-05-24,55.18,55.30,54.13,54.75,19443000.0,INTC
2018-05-25,54.80,55.50,54.54,55.44,18550000.0,INTC


In [12]:
yahoo_MSFT_df

,Open,High,Low,Close,Volume,Symbol
Date,,,,,,
2018-05-15,97.24,97.85,96.34,97.32,24594000.0,MSFT
2018-05-16,97.36,97.40,96.62,97.15,17384700.0,MSFT
2018-05-17,96.76,97.54,95.83,96.18,17246700.0,MSFT
2018-05-18,96.01,96.93,96.01,96.36,17865800.0,MSFT
2018-05-21,97.00,98.01,96.80,97.60,19422500.0,MSFT
2018-05-22,97.68,98.17,97.20,97.50,15441200.0,MSFT
2018-05-23,96.71,98.73,96.32,98.66,21114800.0,MSFT
2018-05-24,98.73,98.94,96.81,98.31,21540800.0,MSFT
2018-05-25,98.30,98.98,97.86,98.36,18363900.0,MSFT


In [13]:
yahoo_ORCL_df

,Open,High,Low,Close,Volume,Symbol
Date,,,,,,
2018-05-15,46.51,46.82,46.25,46.81,11304900.0,ORCL
2018-05-16,46.93,46.93,46.45,46.73,9874200.0,ORCL
2018-05-17,46.69,47.06,46.42,46.54,10162300.0,ORCL
2018-05-18,46.69,46.79,46.24,46.32,13517300.0,ORCL
2018-05-21,46.53,47.29,46.49,47.21,13416300.0,ORCL
2018-05-22,47.24,47.54,46.31,46.37,18505100.0,ORCL
2018-05-23,46.08,46.97,46.02,46.90,8921500.0,ORCL
2018-05-24,46.66,46.94,46.14,46.46,12496400.0,ORCL
2018-05-25,46.49,47.17,46.48,47.00,11354300.0,ORCL


In [14]:
yahoo_UPS_df

,Open,High,Low,Close,Volume,Symbol
Date,,,,,,
2018-05-15,115.78,116.47,114.85,115.61,3480400.0,UPS
2018-05-16,115.54,116.67,115.53,116.32,1970000.0,UPS
2018-05-17,116.59,117.44,116.40,116.59,3697200.0,UPS
2018-05-18,115.67,115.93,115.10,115.38,2450900.0,UPS
2018-05-21,116.00,117.28,115.50,117.08,3333300.0,UPS
2018-05-22,117.01,117.14,115.29,115.36,2027100.0,UPS
2018-05-23,114.85,115.42,113.78,115.28,1974000.0,UPS
2018-05-24,115.38,115.81,114.41,115.63,2297300.0,UPS
2018-05-25,115.49,115.90,115.20,115.49,1583600.0,UPS


In [15]:
yahoo_XOM_df

,Open,High,Low,Close,Volume,Symbol
Date,,,,,,
2018-05-15,81.85,82.03,81.40,81.79,12595700.0,XOM
2018-05-16,81.76,82.05,81.17,82.02,9570000.0,XOM
2018-05-17,82.20,82.28,81.44,81.88,11224100.0,XOM
2018-05-18,81.55,81.80,81.26,81.30,10030900.0,XOM
2018-05-21,81.73,82.35,81.52,82.28,8822400.0,XOM
2018-05-22,82.44,82.65,81.35,81.64,10753400.0,XOM
2018-05-23,80.95,82.23,80.57,82.15,15140900.0,XOM
2018-05-24,81.35,81.36,79.95,80.27,13360500.0,XOM
2018-05-25,79.28,79.33,78.09,78.71,14562000.0,XOM


##### IEX Financial DataFrames with Date Index

In [16]:
iex_BP_df

,Open,High,Low,Close,Volume,Symbol
Date,,,,,,
2018-05-15,46.51,46.9853,46.2700,46.79,7149574.0,BP
2018-05-16,46.80,46.8200,46.4300,46.71,4866498.0,BP
2018-05-17,47.01,47.3700,46.9331,47.19,7207915.0,BP
2018-05-18,47.30,47.4350,47.1442,47.38,4276638.0,BP
2018-05-21,47.38,47.8300,47.3800,47.79,5412095.0,BP
2018-05-22,47.37,47.7050,46.9300,47.09,7979923.0,BP
2018-05-23,46.13,46.5100,45.9000,46.45,7871671.0,BP
2018-05-24,45.67,45.9400,45.4000,45.75,7917840.0,BP
2018-05-25,44.82,44.8300,44.0900,44.35,9171285.0,BP


In [17]:
iex_FDX_df

,Open,High,Low,Close,Volume,Symbol
Date,,,,,,
2018-05-15,248.8226,248.9224,245.6011,246.6883,1638203.0,FDX
2018-05-16,247.1271,248.0627,245.8904,246.4688,1101189.0,FDX
2018-05-17,247.0373,249.5706,246.6287,247.8053,1083981.0,FDX
2018-05-18,247.2268,249.0720,246.0599,248.7927,1195706.0,FDX
2018-05-21,250.3386,254.2682,250.0195,253.9890,1384222.0,FDX
2018-05-22,254.4777,255.6247,251.6851,251.9244,1011323.0,FDX
2018-05-23,251.3559,251.3559,247.9649,249.8100,1106812.0,FDX
2018-05-24,249.5208,251.3460,247.8452,251.0767,1496280.0,FDX
2018-05-25,250.1990,252.5129,250.1990,250.8373,841197.0,FDX


In [18]:
iex_INTC_df

,Open,High,Low,Close,Volume,Symbol
Date,,,,,,
2018-05-15,54.58,54.7500,53.560,53.92,21712649.0,INTC
2018-05-16,54.17,54.7000,53.955,54.64,16260459.0,INTC
2018-05-17,54.55,55.4400,54.420,54.81,16872598.0,INTC
2018-05-18,54.53,54.5400,53.470,53.50,23679944.0,INTC
2018-05-21,54.36,55.2200,53.860,54.32,25367377.0,INTC
2018-05-22,54.73,55.0500,54.050,54.45,19009588.0,INTC
2018-05-23,54.00,55.2300,53.840,55.21,20555425.0,INTC
2018-05-24,55.18,55.3000,54.130,54.75,19667095.0,INTC
2018-05-25,54.80,55.5000,54.540,55.44,18549989.0,INTC


In [19]:
iex_MSFT_df

,Open,High,Low,Close,Volume,Symbol
Date,,,,,,
2018-05-15,96.8203,97.4277,95.9242,96.90,24594010.0,MSFT
2018-05-16,97.3600,97.4000,96.6150,97.15,17384742.0,MSFT
2018-05-17,96.7600,97.5401,95.8300,96.18,17246716.0,MSFT
2018-05-18,96.0100,96.9300,96.0100,96.36,17865840.0,MSFT
2018-05-21,97.0000,98.0100,96.8000,97.60,19422467.0,MSFT
2018-05-22,97.6800,98.1700,97.2000,97.50,15441189.0,MSFT
2018-05-23,96.7100,98.7300,96.3200,98.66,21251222.0,MSFT
2018-05-24,98.7250,98.9400,96.8100,98.31,26649287.0,MSFT
2018-05-25,98.3000,98.9800,97.8600,98.36,18363918.0,MSFT


In [20]:
iex_ORCL_df

,Open,High,Low,Close,Volume,Symbol
Date,,,,,,
2018-05-15,46.51,46.820,46.2500,46.81,11304873.0,ORCL
2018-05-16,46.93,46.930,46.4500,46.73,9874195.0,ORCL
2018-05-17,46.69,47.055,46.4200,46.54,10162341.0,ORCL
2018-05-18,46.69,46.790,46.2400,46.32,13517330.0,ORCL
2018-05-21,46.53,47.285,46.4900,47.21,13416333.0,ORCL
2018-05-22,47.24,47.540,46.3100,46.37,18505113.0,ORCL
2018-05-23,46.08,46.970,46.0200,46.90,9204379.0,ORCL
2018-05-24,46.66,46.940,46.1400,46.46,12497658.0,ORCL
2018-05-25,46.49,47.165,46.4800,47.00,11354337.0,ORCL


In [21]:
iex_UPS_df

,Open,High,Low,Close,Volume,Symbol
Date,,,,,,
2018-05-15,114.8763,115.5610,113.9536,114.7077,3480370.0,UPS
2018-05-16,114.6382,115.7544,114.6283,115.4121,1970011.0,UPS
2018-05-17,115.6800,116.5234,115.4915,115.6800,3697242.0,UPS
2018-05-18,115.6700,115.9300,115.1000,115.3800,2450899.0,UPS
2018-05-21,116.0000,117.2800,115.5000,117.0800,3333298.0,UPS
2018-05-22,117.0100,117.1400,115.2900,115.3600,2027096.0,UPS
2018-05-23,114.8500,115.4200,113.7800,115.2800,2013573.0,UPS
2018-05-24,115.3800,115.8100,114.4100,115.6300,2303415.0,UPS
2018-05-25,115.4900,115.9000,115.2000,115.4900,1583600.0,UPS


In [22]:
iex_XOM_df

,Open,High,Low,Close,Volume,Symbol
Date,,,,,,
2018-05-15,81.85,82.0300,81.400,81.79,12595736.0,XOM
2018-05-16,81.76,82.0450,81.170,82.02,9569958.0,XOM
2018-05-17,82.20,82.2800,81.440,81.88,11224123.0,XOM
2018-05-18,81.55,81.8000,81.260,81.30,10030931.0,XOM
2018-05-21,81.73,82.3500,81.520,82.28,8822364.0,XOM
2018-05-22,82.44,82.6500,81.350,81.64,10753428.0,XOM
2018-05-23,80.95,82.2300,80.570,82.15,15140896.0,XOM
2018-05-24,81.35,81.3600,79.950,80.27,13360454.0,XOM
2018-05-25,79.28,79.3300,78.090,78.71,14561988.0,XOM
